# Liftover tools pipeline(hg38>hg19)

Stept 0: 會下載Picard tools, hg19, hg38ToHg19的liftover file，並且做前處理

Stept 1: 選擇VCF資料夾，流程會掃過資料夾中所有VCF，另外選擇liftover file及參考序列

Stept 2: 執行Picard tools LiftoverVcf功能

## Stept 0: 如果你不是第一次跑這個流程，可以跳過這個流程到Step 1

In [ ]:
try:
    import wget
except:
    !pip install wget

try:
    import ipyfilechooser
except:
    !pip install ipyfilechooser

import os 
import gzip
dir_path = os.getcwd()

#Download reference seq
filename = dir_path + '/' + os.path.basename('https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz')
if os.path.exists(filename):
    print('hg19.fa.gz 已下載')
else :
    wget.download('https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz') 
    g = gzip.GzipFile(mode="rb", fileobj=open('hg19.fa.gz' , 'rb')) # python gzip 解壓
    open(r"hg19.fa", "wb").write(g.read())
    
#Download liftover file
filename = dir_path + '/' + os.path.basename('https://hgdownload.soe.ucsc.edu/goldenPath/hg38/liftOver/hg38ToHg19.over.chain.gz')
if os.path.exists(filename):
    print('hg38ToHg19.over.chain.gz 已下載')
else :
    wget.download('https://hgdownload.soe.ucsc.edu/goldenPath/hg38/liftOver/hg38ToHg19.over.chain.gz') 
    g = gzip.GzipFile(mode="rb", fileobj=open('hg38ToHg19.over.chain.gz' , 'rb')) # python gzip 解壓
    open(r"hg38ToHg19.over.chain", "wb").write(g.read())

#Download picard tools
filename = dir_path + '/' + os.path.basename('https://github.com/broadinstitute/picard/releases/download/2.23.8/picard.jar')
if os.path.exists(filename):
    print('picard.jar 已下載')
else :
    wget.download('https://github.com/broadinstitute/picard/releases/download/2.23.8/picard.jar') 


 13% [.........                                                             ] 129515520 / 948731419

### 確認Java是否安裝，如未安裝 請安裝JDK https://www.oracle.com/tw/java/technologies/javase-downloads.html

In [ ]:
!java -version

### 產生fasta.dict檔

In [ ]:
if os.path.exists("hg19.dict"):
    print('CreateSequenceDictionary已完成, 請至下個步驟')
else:
    !java -jar .\picard.jar CreateSequenceDictionary -R "hg19.fa" 

## Stept 1: 選擇檔案

In [ ]:
from ipyfilechooser import FileChooser
# Create and display a FileChooser widget
print("選擇要掃描的VCF資料夾")
vcf = FileChooser(dir_path)
vcf.show_only_dirs = True
display(vcf)

print("Select Chain file[.chain file]")
chain = FileChooser(dir_path)
chain.show_only_dirs = False
display(chain)

print("Select reference file[.fa file]")
fasta = FileChooser(dir_path)
fasta.show_only_dirs = False
display(fasta)


## Stept 2: 執行 picard liftoverVcf  

In [ ]:
#output file path as _liftover.vcf
if os.path.exists(vcf.selected_path+"\\"+"liftover"):
    print("liftover資料夾已建立")
else:
    !mkdir liftover

### LiftoverVcf

In [ ]:
print(vcf.selected)
print(chain.selected)
print(fasta.selected)
files = os.listdir(vcf.selected)
for i in files:
    if 'vcf' in i:
        liftover_vcf=vcf.selected_path+"\\"+"liftover"+"\\"+i.replace(".vcf", "_liftover.vcf")
        reject_vcf=vcf.selected_path+"\\"+"liftover"+"\\"+i.replace(".vcf", "_rejected_variants.vcf")
        input=vcf.selected_path+'\\'+i
        !java -Xmx8G -jar .\picard.jar LiftoverVcf -I $input -O $liftover_vcf  --CHAIN $chain.selected --REJECT $reject_vcf -R $fasta.selected

### 確認 liftovervcf 是否完成

In [ ]:
files = os.listdir(vcf.selected+"liftover")
for i in files:
    print(i)